In [122]:
import os
import shutil
import pandas as pd
import pyodbc

# SQL Server connection parameters
server = 'SUDHISAMI\\SUDHISAMI'
database = 'Python_DB'
username = 'newsa'
password = 'Password123*'
driver = '{ODBC Driver 17 for SQL Server}'

# Establish the connection
conn = pyodbc.connect(
    f'DRIVER={driver};SERVER={server};DATABASE={database};UID={username};PWD={password}'
)
cursor = conn.cursor()

# Directory containing the Excel files
directory = r'C:\Users\junea\Documents\Files\data'  # Path to the folder containing sales files
archive_directory = r'C:\Users\junea\Documents\Files\data\Archive'  # Path to the archive folder

# Ensure the archive directory exists
if not os.path.exists(archive_directory):
    os.makedirs(archive_directory)

# Loop through each file in the directory
for filename in os.listdir(directory):
    if filename.startswith('Sales_') and filename.endswith('.xlsx'):
        file_path = os.path.join(directory, filename)
        try:
            # Load the Excel file into a DataFrame
            df = pd.read_excel(file_path, engine='openpyxl')
            # Add a new column for the file name
            df['source_file'] = filename
        except Exception as e:
            print(f"Error reading {file_path}: {e}")
            continue
        
        # Check if the DataFrame is not empty
        if not df.empty:
            # Define the insert query
            insert_query = '''
            INSERT INTO SalesFact (sales_id, date_id, customer_id, product_id, store_id, sales_amount, quantity_sold, discount_amount, total_amount, source_file)
            VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
            '''

            # Insert each row into the SQL Server table
            for index, row in df.iterrows():
                cursor.execute(insert_query, 
                               int(row['sales_id']), 
                               int(row['date_id']), 
                               int(row['customer_id']), 
                               int(row['product_id']), 
                               int(row['store_id']), 
                               float(row['sales_amount']), 
                               int(row['quantity_sold']), 
                               float(row['discount_amount']), 
                               float(row['total_amount']),
                               row['source_file'])

            # Commit the transaction
            conn.commit()
            
            # Move the file to the archive directory
            archive_path = os.path.join(archive_directory, filename)
            shutil.move(file_path, archive_path)
            print(f"Moved {file_path} to {archive_path}")
        else:
            print(f"No data found in {file_path}")

# Close the connection
cursor.close()
conn.close()


No data found in C:\Users\junea\Documents\Files\data\Sales_2021.xlsx
Moved C:\Users\junea\Documents\Files\data\Sales_2022.xlsx to C:\Users\junea\Documents\Files\data\Archive\Sales_2022.xlsx
Moved C:\Users\junea\Documents\Files\data\Sales_2023.xlsx to C:\Users\junea\Documents\Files\data\Archive\Sales_2023.xlsx
No data found in C:\Users\junea\Documents\Files\data\Sales_2024.xlsx


In [121]:
os.listdir(directory)

['Archive',
 'Product Load into SQL Server.py',
 'Python Joins and graphs.ipynb',
 'Python Pandas ETL 2.docx',
 'Python Pandas ETL 2.pdf',
 'Python Script ETL 2.py',
 '___Sales_2022.xlsx']

In [110]:
filename

'Sales_2021.xlsx'